In [1]:
#imports
import numpy as np
import pandas as pd
import random
from decpomdp import DecPOMDP
import Classes
# import tensorflow.python.keras as keras
import matplotlib.pyplot as plt





## Import Problem

In [3]:
horizon = 2
prob = DecPOMDP("dectiger", horizon)
prob.reset()
Classes.set_problem(prob)

problem set to dectiger


In [4]:
game = Classes.PBVI(problem=prob,horizon=horizon,density=1,gametype="stackelberg")
game.belief_space.expansion()

	belief expansion done


#### 7. Define belief update function

In [ ]:
def next_belief(b0,a0,z):
    # returns the value of b1
    b1 = []

    if type(z) != int :
        z = observations.index(z)

    for x1 in states:
        val = 0
        for x0 in states:
            for u in joint_actions:
                val += b0[x0] * a0[u] * prob.transition_fn[u][x0][x1]  * prob.observation_fn[u][x0][z]
        # print(f"value = {val}")
        b1.append(val)
    b1 = normalize(b1)
    if np.sum(b1)<= 1 and np.sum(b1)> 0.99999:
        return b1
    else:
        print("err0r : belief doesn not sum up to 1\n")
        print(b1)
    return b1

a0 = generate_sample_joint_actions(1)[0]
b0 = [0.4,0.6]
next_belief(b0,a0,1)

    

array([0.49817137, 0.50182863])

### 8. Define linear program

In [ ]:
#define functions for subroutine to be stored in dictionary
def V_aj_1(u1,u2,b):
    return b[0] * joint_reward[0][u1][u2] + b[1] * joint_reward[1][u1][u2]

def V_aj_2(u1,u2,b):
    return b[0] * joint_reward[0][u1][u2] + b[1] * joint_reward[1][u1][u2]



#define this in the program later with variables inside the linear program
def V1_ai(u1,b):
    values = []
    for u2 in actions :
        values.append(b[0] * joint_reward[0][u1][u2] + b[1] * joint_reward[1][u1][u2])
    return values


In [ ]:
from docplex.mp.model import Model
def V(b):
    subroutine_joint = {}
    subroutine_single = {}
    for i in actions:
        for j in actions:
            str1 = f"a0_{i}{j}"
            subroutine_joint[str1] = [V_aj_1(i,j,b),V_aj_2(i,j,b)]
            
    for j in actions:    
        str2 = f"a1_{j}"
        subroutine_single[str2] = V1_ai(j,b)



    milp = Model("tiger problem")
    a1_0,a1_1,a1_2 = milp.continuous_var_list(3,name = ["a1_0","a1_1","a1_2"],ub=1,lb=0)
    a2_0,a2_1,a2_2 = milp.binary_var_list(3, name = ["a2_0","a2_1","a2_2"],ub=1)
    a0_00, a0_01 , a0_02, a0_10, a0_11 , a0_12, a0_20, a0_21 , a0_22 = milp.continuous_var_list(9,name  = [f"a0_{i}{j}" for i in actions for j in actions],ub=1,lb=0)

    # define objective function
    # obj_fn = a0_00*subroutine["a0_00"][0] + a0_01*subroutine["a0_01"][0] + a0_02*subroutine["a0_02"][0] + a0_10*subroutine["a0_10"][0] + a0_11*subroutine["a0_11"][0] + a0_12*subroutine["a0_12"][0] + a0_20*subroutine["a0_20"][0] + a0_21*subroutine["a0_21"][0] + a0_22*subroutine["a0_22"][0]

    # define objective function V^1(b0) = max(a_0) summation[x]summation[u]a_0(u)b(x) r^1(x,u)
    obj_fn = 0
    for i in subroutine_joint.keys():
        if type(subroutine_joint[i]) == list:
            # print(i)
            obj_fn += subroutine_joint[i][0] * milp.get_var_by_name(str(i))
    milp.maximize(obj_fn)

    # define constraints 

    # define lhs of linear equivalence expression equal to :
    # V^2(b0,a1,a2)

    lhs = 0
    for a_ii in subroutine_joint.keys():
        lhs += subroutine_joint[a_ii][1] * milp.get_var_by_name(a_ii)

    for u2 in actions:    
        rhs = 0       
        for a1_i in subroutine_single.keys():
            rhs += subroutine_single[a1_i][u2] * milp.get_var_by_name(a1_i)
        milp.add_constraint(lhs>=rhs)
        


    #add seperability constraints
    milp.add_constraint(a0_00 + a0_01 + a0_02 == a1_0)
    milp.add_constraint(a0_10 + a0_11 + a0_12 == a1_1)
    milp.add_constraint(a0_20 + a0_21 + a0_22 == a1_2)
    milp.add_constraint(a0_00 + a0_10 + a0_20 == a2_0)
    milp.add_constraint(a0_01 + a0_11 + a0_21 == a2_1)
    milp.add_constraint(a0_02 + a0_12 + a0_22 == a2_2)
    milp.add_constraint(a0_00 + a0_01 + a0_02+ a0_10+ a0_11 + a0_12+ a0_20+ a0_21 + a0_22 ==1)

    #add constraint for joint action to sum to 1 

    sol = milp.solve()
    # print("SOLUTION : ")
    # print(f"joint actions : {milp.solution.get_values([a0_00,a0_01,a0_02,a0_10,a0_11,a0_12,a0_20,a0_21,a0_22])}")
    # print(f"a1 : {milp.solution.get_values([a1_0,a1_1,a1_2])}")
    # print(f"a2 : {milp.solution.get_values([a2_0,a2_1,a2_2])}")

    
    print(f"value solution = {milp.solution.get_objective_value()}")
    # redo calculations with competitive reward matrix by hand 


    # milp.write("test.lp")

    # milp.export_as_lp("tigerProblem")
    return milp.solution.get_objective_value()

# I. Implementation of PBVI for cooperative tiger game

## Sample Tree

In [ ]:
def check_min_distance(b_t ,depth, delta,Beliefs):
    """function to check if a new belief point is "sufficiently different from other points in the bag of beliefs """
    b = np.array(Beliefs[f"B{depth}"])
    min_b_old= min(b, key=lambda x: np.linalg.norm(x-b_t))
    min_magnitude = np.linalg.norm(min_b_old-b_t)
    
    if min_magnitude > delta :
        # print(min_b_old , min_magnitude)
        print("\nB{} : {} ".format(depth,Beliefs[f"B{depth}"]))
        return True 

def check_min_distance2(b_t ,depth, delta,Beliefs):
    """function to check if a new belief point is "sufficiently different from other points in the bag of beliefs """
    b = np.array([i for i in Beliefs[depth]])
    print(b)
    min_b_old= min(b, key=lambda x: np.linalg.norm(x-b_t))
    min_magnitude = np.linalg.norm(min_b_old-b_t)
    
    if min_magnitude > delta :
        # print(min_b_old , min_magnitude)
        print("\nB{} : {} ".format(depth,Beliefs[depth]))
        return True 
  


In [ ]:
# n = 10
Beliefs = {"B0" : [[0.5,0.5]]}


#samples joint DR from an identity matrix so that each DR = deterministic U
def_actions = [i for i in np.identity(9)]

gamma = 0.5
Q_table = {}
delta = 0.001
for t in range(horizon):
    Beliefs[f"B{t+1}"]= []
    for b_t in Beliefs[f"B{t}"]:
        for uj in def_actions:
            for z in joint_observations:
                b_t_1 = next_belief(b_t,uj,z)
            
                if not Beliefs[f"B{t+1}"] :
                    Beliefs[f"B{t+1}"].append(b_t_1)

                elif check_min_distance(b_t_1,t+1,delta,Beliefs):
                    Beliefs[f"B{t+1}"].append(b_t_1)

print(f"Depth of tree : {len(Beliefs)}")
Beliefs

        



B1 : [array([0.96979866, 0.03020134])] 

B1 : [array([0.96979866, 0.03020134]), array([0.5, 0.5])] 

B2 : [array([9.99031124e-01, 9.68876343e-04])] 

B2 : [array([9.99031124e-01, 9.68876343e-04]), array([0.96979866, 0.03020134])] 

B2 : [array([9.99031124e-01, 9.68876343e-04]), array([0.96979866, 0.03020134]), array([0.5, 0.5])] 

B2 : [array([9.99031124e-01, 9.68876343e-04]), array([0.96979866, 0.03020134]), array([0.5, 0.5]), array([0.03020134, 0.96979866])] 

B3 : [array([9.99969799e-01, 3.02009694e-05])] 

B3 : [array([9.99969799e-01, 3.02009694e-05]), array([9.99031124e-01, 9.68876343e-04])] 

B3 : [array([9.99969799e-01, 3.02009694e-05]), array([9.99031124e-01, 9.68876343e-04]), array([0.96979866, 0.03020134])] 

B3 : [array([9.99969799e-01, 3.02009694e-05]), array([9.99031124e-01, 9.68876343e-04]), array([0.96979866, 0.03020134]), array([0.5, 0.5])] 

B3 : [array([9.99969799e-01, 3.02009694e-05]), array([9.99031124e-01, 9.68876343e-04]), array([0.96979866, 0.03020134]), array([

{'B0': [[0.5, 0.5]],
 'B1': [array([0.96979866, 0.03020134]),
  array([0.5, 0.5]),
  array([0.03020134, 0.96979866])],
 'B2': [array([9.99031124e-01, 9.68876343e-04]),
  array([0.96979866, 0.03020134]),
  array([0.5, 0.5]),
  array([0.03020134, 0.96979866]),
  array([9.68876343e-04, 9.99031124e-01])],
 'B3': [array([9.99969799e-01, 3.02009694e-05]),
  array([9.99031124e-01, 9.68876343e-04]),
  array([0.96979866, 0.03020134]),
  array([0.5, 0.5]),
  array([0.03020134, 0.96979866]),
  array([9.68876343e-04, 9.99031124e-01]),
  array([3.02009694e-05, 9.99969799e-01])],
 'B4': [array([9.99999059e-01, 9.40542140e-07]),
  array([9.99031124e-01, 9.68876343e-04]),
  array([0.5, 0.5]),
  array([0.96979866, 0.03020134]),
  array([0.03020134, 0.96979866]),
  array([9.68876343e-04, 9.99031124e-01]),
  array([3.02009694e-05, 9.99969799e-01])],
 'B5': [array([9.99999971e-01, 2.92902663e-08]),
  array([0.5, 0.5]),
  array([9.99031124e-01, 9.68876343e-04]),
  array([0.96979866, 0.03020134]),
  array([

## Solving tree

#### 1. define Q functions for Cooperative Game

In [ ]:
def value_closest_b(bt,V_table,t):
    max = np.inf
    for bt_next in V_table[t].keys():
        distance = np.abs(np.linalg.norm(bt - np.array(bt_next)))
        if distance<max: 
            next = bt_next
            max = distance
    print(next)
    return V_table[t][next][0]

def  Q(bt,t,u,V_table):
    """"""
    # V_next_t = V_table[t+1]
    sum = 0
    for x in states :
        for z in joint_observations :
            if t == horizon:
                V_next_t =  0
            else:
                b_next = next_belief(bt,np.identity(9)[u],z)
                try:
                    print(V_next_t)
                    V_next_t = V_table[t+1][tuple(b_next)][0]
                    
                except:
                    V_next_t = value_closest_b(b_next,V_table,t+1)
                     
            sum += bt[x] * prob.reward_fn_sa[u][x] + gamma * Pr_z(z,bt,u) * V_next_t
    return sum

def Q_vectors(bt,t,u,V_table):
  
    V_next_t = V_table[t+1]
    Q_vector = []
  
    for x in states :
        sum = 0
        for z in joint_observations :
            if t == horizon:
                V_next_t =  0
            else:
                b_next = next_belief(bt,np.identity(9)[u],z)
                try:
                    V_next_t = V_table[t+1][tuple(b_next)][0]
                except:
                    V_next_t = value_closest_b(b_next,V_table,t+1)
 
            sum += bt[x] * prob.reward_fn_sa[u][x] + gamma * Pr_z(z,bt,u) * V_next_t
        Q_vector.append(sum)
    return Q_vector

    


In [ ]:
def generate_probability_distribution(length):
    # Generate random probabilities
    probabilities = np.random.rand(length)

    # Normalize to make the sum equal to 1
    probabilities /= probabilities.sum()

    return probabilities

#### 2. define Q functions for Competitive Game

In [ ]:
def Qcomp_1(bt,t,u,V_table):
    """"""
    V_next_t = V_table[t+1]

    sum = 0
    for x in states :
        for z in joint_observations :
            if t == horizon:
                V_next_t =  0
            else:
                b_next = next_belief(bt,np.identity(9)[u],z)
                try:
                    V_next_t = V_table[t+1][tuple(b_next)][0]
                except:
                    V_next_t = value_closest_b(b_next,V_table,t+1)
 
            sum += bt[x] * comp_reward_1[u][x] + gamma * Pr_z(z,bt,u) * V_next_t
    return sum

def Qcomp_2(bt,t,u,V_table):
    """"""
    V_next_t = V_table[t+1]
   
    sum = 0
    for x in states :
        for z in joint_observations :
                if t == horizon:
                    V_next_t =  0
                else:
                    b_next = next_belief(bt,np.identity(9)[u],z)
                    try:
                        V_next_t = V_table[t+1][tuple(b_next)][0]
                    except:
                        V_next_t = value_closest_b(b_next,V_table,t+1)
 
                sum += bt[x] * comp_reward_2[u][x] + gamma * Pr_z(z,bt,u) * V_next_t
    return sum

def Q1_vectors(bt,t,u,V_table):
  
    V_next_t = V_table[t+1]
  
    
    Q_vector = []
  
    for x in states :
        sum = 0
        for z in joint_observations :
            if t == horizon:
                V_next_t =  0
            else:
                b_next = next_belief(bt,np.identity(9)[u],z)
                try:
                    V_next_t = V_table[t+1][tuple(b_next)]
                except:
                    V_next_t = value_closest_b(b_next,V_table,t+1)
 
            sum += bt[x] * prob.reward_fn_sa[u][x] + gamma * Pr_z(z,bt,u) * V_next_t
        Q_vector.append(sum)
    return Q_vector

def Q2_vectors(bt,t,u,V_table):
  
    V_next_t = V_table[t+1]
    try:
        V_next_t = V_next_t[tuple(bt)]
        # print(f"V_{t+1} = {V_next_t}")
    except:
        print("last horizon")
        V_next_t =  0
    
    Q_vector = []
  
    for x in states :
        sum = 0
        for z in joint_observations :
            try :
                if t >= horizon:
                    V_next_t =  0
                else:
                    b_next = next_belief(bt,np.identity(9)[u],z)
                    V_next_t = V_next_t[tuple(b_next)]
            except:
                print("err0r : no value found for V(next belief)")
                return
            sum += bt[x] * prob.reward_fn_sa[u][x] + gamma * Pr_z(z,bt,u) * V_next_t
        Q_vector.append(sum)
    return Q_vector







#### 2. define Linear Program for Cooperative game


In [ ]:
from docplex.mp.model import Model
def LP_coop(Q):
 
    milp = Model("tiger problem")
    a1_0 , a1_1, a1_2 = milp.continuous_var_list(3,name = ["a1_0","a1_1","a1_2"],ub=1,lb=0)
    a2_0 , a2_1, a2_2 = milp.binary_var_list(3, name = ["a2_0","a2_1","a2_2"],ub=1)
    a0_00,a0_01,a0_02,a0_10,a0_11,a0_12,a0_20,a0_21,a0_22 = milp.continuous_var_list(9,name  = [f"a0_{i}{j}" for i in actions for j in actions],ub=1,lb=0)
    
    joint_DR = [a0_00,a0_01,a0_02,a0_10,a0_11,a0_12,a0_20,a0_21,a0_22]
    player1_DR = [a1_0 , a1_1, a1_2]


    # define objective function 
    obj_fn = 0
    for idx,a_ii in enumerate(joint_DR):
                obj_fn += Q[idx]  * a_ii
    milp.maximize(obj_fn)


    #
    lhs = 0
    for idx,a_ii in enumerate(joint_DR):
            lhs += Q[idx] * a_ii 


    for u2 in actions:    
        rhs = 0       
        for a1_i in player1_DR:
            for x in states:
                rhs += Q[u2] * a1_i 
            u2 += 3 # used to select joint actions where u2 is consistent
        milp.add_constraint(lhs>=rhs)
        



  

    #add seperability constraints
    milp.add_constraint(a0_00 + a0_01 + a0_02 == a1_0)
    milp.add_constraint(a0_10 + a0_11 + a0_12 == a1_1)
    milp.add_constraint(a0_20 + a0_21 + a0_22 == a1_2)
    milp.add_constraint(a0_00 + a0_10 + a0_20 == a2_0)
    milp.add_constraint(a0_01 + a0_11 + a0_21 == a2_1)
    milp.add_constraint(a0_02 + a0_12 + a0_22 == a2_2)
    milp.add_constraint(a0_00 + a0_01 + a0_02+ a0_10+ a0_11 + a0_12+ a0_20+ a0_21 + a0_22 ==1)

    sol = milp.solve()
    # print("SOLUTION : ")
    # print(f"joint actions : {milp.solution.get_values([a0_00,a0_01,a0_02,a0_10,a0_11,a0_12,a0_20,a0_21,a0_22])}")
    # print(f"a1 : {milp.solution.get_values([a1_0,a1_1,a1_2])}")
    # print(f"a2 : {milp.solution.get_values([a2_0,a2_1,a2_2])}")

    print(f"value solution = {milp.solution.get_objective_value()}")

    # milp.write("test.lp")
    # milp.export_as_lp("tigerProblem")
    return milp.solution.get_objective_value(),milp.solution.get_values(joint_DR)



# rew = [[prob.reward_fn_sa[i][1] for i in range(9)] for i in range(2)]
# LP([0.5,0.5],rew)
    

In [ ]:
from docplex.mp.model import Model
def LP_comp(Q1,Q2):
 
    milp = Model("tiger problem")
    a1_0 , a1_1, a1_2 = milp.continuous_var_list(3,name = ["a1_0","a1_1","a1_2"],ub=1,lb=0)
    a2_0 , a2_1, a2_2 = milp.binary_var_list(3, name = ["a2_0","a2_1","a2_2"],ub=1)
    a0_00,a0_01,a0_02,a0_10,a0_11,a0_12,a0_20,a0_21,a0_22 = milp.continuous_var_list(9,name  = [f"a0_{i}{j}" for i in actions for j in actions],ub=1,lb=0)
    
    joint_DR = [a0_00,a0_01,a0_02,a0_10,a0_11,a0_12,a0_20,a0_21,a0_22]
    player1_DR = [a1_0 , a1_1, a1_2]


    # define objective function 
    obj_fn = 0
    for idx,a_ii in enumerate(joint_DR):
            obj_fn += Q1[idx]  * a_ii
    milp.maximize(obj_fn)

    # define constraints 

    # define lhs of linear equivalence expression equal to :
    # V^2(b0,a1,a2)

    lhs = 0
    for idx,a_ii in enumerate(joint_DR):
            lhs += Q2[idx] * a_ii 

    # define rhs of linear equivalence expression equal to :
    # V^2(b0,a1,a2)
    for u2 in actions:    
        rhs = 0       
        for a1_i in player1_DR:
                rhs += Q2[u2] * a1_i 
                u2 += 3 # used to select index of joint actions where u2 is consistent
        milp.add_constraint(lhs>=rhs)
        


    #add seperability constraints
    milp.add_constraint(a0_00 + a0_01 + a0_02 == a1_0)
    milp.add_constraint(a0_10 + a0_11 + a0_12 == a1_1)
    milp.add_constraint(a0_20 + a0_21 + a0_22 == a1_2)
    milp.add_constraint(a0_00 + a0_10 + a0_20 == a2_0)
    milp.add_constraint(a0_01 + a0_11 + a0_21 == a2_1)
    milp.add_constraint(a0_02 + a0_12 + a0_22 == a2_2)
    milp.add_constraint(a0_00 + a0_01 + a0_02+ a0_10+ a0_11 + a0_12+ a0_20+ a0_21 + a0_22 ==1)

    sol = milp.solve()
    # print("SOLUTION : ")
    # print(f"joint actions : {milp.solution.get_values([a0_00,a0_01,a0_02,a0_10,a0_11,a0_12,a0_20,a0_21,a0_22])}")
    # print(f"a1 : {milp.solution.get_values([a1_0,a1_1,a1_2])}")
    # print(f"a2 : {milp.solution.get_values([a2_0,a2_1,a2_2])}")

    print(f"value solution = {milp.solution.get_objective_value()}")

    # milp.write("test.lp")
    # milp.export_as_lp("tigerProblem")
    return milp.solution.get_objective_value(),milp.solution.get_values(joint_DR)



# rew = [[prob.reward_fn_sa[i][1] for i in range(9)] for i in range(2)]
# LP([0.5,0.5],rew)
    

#### 3. Solve for Coop Game

In [ ]:

# Q table initialized as dictionary with "t" as key
# dictionary within dictionary 

Q_table= {}
V_table = {horizon+1 : 0}

for t in range(horizon,-1,-1):
    print(f"====== Timestep : {t} ======")
    V_table[t] = dict()
    Q_table[t] = dict()
    for bt in Beliefs[f"B{t}"]:
        q_val = []
        q_vectors = []
       
        # calculate Q-val of each joint action and keep q vectors to use in the Linear program
        for u in joint_actions:
            q_val.append(Q(bt,t,u,V_table))
            q_vectors.append(Q_vectors(bt,t,u,V_table))
        
       # save Q(b,a) and q_vectors in Q table
        Q_table[t][tuple(bt)] = q_val

        # Calculate linear program of t-1
        Vt_b , a_optimal = LP_coop(q_val) # implement linear program
        
        #add V_t(b) to V table
        V_table[t][tuple(bt)] = (Vt_b ,a_optimal)
    print(f"==== end of Timestep {t} =======\n")
    print(f"V[{t}] : {V_table[t]}")
# print(V_table)
# print(Q_table)
pd.DataFrame.from_dict(V_table, orient='index')


====== Timestep : 5 ======
value solution = -1.4210854715202004e-14
value solution = -8.0
value solution = 0.0
value solution = 2.842170943040401e-14
value solution = 2.1316282072803006e-14
value solution = 1.4210854715202004e-14
value solution = 1.4210854715202004e-14
==== end of Timestep 5 =======

V[5] : {(0.9999999707097337, 2.9290266333289847e-08): (-1.4210854715202004e-14, [0, 0, 0, 0, 0, 0.16666665242834255, 0, 0, 0.8333333475716574]), (0.5, 0.5): (-8.0, [1.0, 0, 0, 0, 0, 0, 0, 0, 0]), (0.9990311236573287, 0.0009688763426712272): (0.0, [0, 0, 0, 0, 0, 0.1661954187723219, 0, 0, 0.8338045812276781]), (0.9697986577181208, 0.030201342281879186): (2.842170943040401e-14, [0, 0, 0, 0, 0, 0.15172217477939076, 0, 0, 0.8482778252206092]), (0.030201342281879193, 0.9697986577181209): (2.1316282072803006e-14, [0, 0, 0, 0, 0.8482778252206091, 0, 0, 0.15172217477939082, 0]), (0.0009688763426712277, 0.9990311236573288): (1.4210854715202004e-14, [0, 0, 0, 0, 0.8338045812276781, 0, 0, 0.166195418

value solution = -10.69285099999999
(0.5, 0.5)
-10.560199999999991
(0.030201342281879193, 0.9697986577181209)
7.105427357601002e-15
(0.030201342281879193, 0.9697986577181209)
7.105427357601002e-15
0.0
-10.560199999999991
(0.030201342281879193, 0.9697986577181209)
7.105427357601002e-15
(0.030201342281879193, 0.9697986577181209)
7.105427357601002e-15
(0.030201342281879193, 0.9697986577181209)
(0.030201342281879193, 0.9697986577181209)
(0.030201342281879193, 0.9697986577181209)
(0.030201342281879193, 0.9697986577181209)
(0.5, 0.5)
-10.560199999999991
-10.560199999999991
-10.560199999999991
-10.560199999999991
-10.560199999999991
-10.560199999999991
-10.560199999999991
(0.5, 0.5)
-10.560199999999991
-10.560199999999991
-10.560199999999991
-10.560199999999991
-10.560199999999991
-10.560199999999991
-10.560199999999991
(0.5, 0.5)
-10.560199999999991
-10.560199999999991
-10.560199999999991
-10.560199999999991
-10.560199999999991
-10.560199999999991
-10.560199999999991
(0.5, 0.5)
-10.560199999

,0
6,0
5,"{(0.9999999707097337, 2.9290266333289847e-08):..."
4,"{(0.9999990594578604, 9.405421396307146e-07): ..."
3,"{(0.9999697990305696, 3.020096943040473e-05): ..."
2,"{(0.9990311236573288, 0.0009688763426712277): ..."
1,"{(0.9697986577181209, 0.030201342281879193): (..."
0,"{(0.5, 0.5): (-10.735302636274996, [1.0, 0, 0,..."


### 4. Solve for Competitive Game

In [ ]:
# Q table initialized as dictionary with "t" as key
# dictionary within dictionary 

Q_comp = {}
V_comp = {horizon+1 : 0}

for t in range(horizon,0,-1):
    print(f"======Timestep : {t}======")
    V_comp[t] = dict()
    Q_comp[t] = dict()
    for bt in Beliefs[f"B{t}"]:
        q1 = []
        q2 = []
        q1_vectors = []
        q2_vectors = []
       
        # calculate Q-val of each joint action and keep q vectors to use in the Linear program
        for u in joint_actions:
            q1.append(Qcomp_1(bt,t,u,V_comp))
            q2.append(Qcomp_2(bt,t,u,V_comp))
            # q1_vectors.append(Q1_vectors(bt,t,u,V_comp))
            # q2_vectors.append(Q2_vectors(bt,t,u,V_comp))
        
       # save Q(b,a) and q_vectors in Q table
        # Q_comp[t][tuple(bt)] = [q1_vectors,q2_vectors]

        # Calculate linear program of t-1
        Vt_b , a_optimal = LP_comp(q1,q2) # implement linear program
        
        #add V_t(b) to V table
        V_comp[t][tuple(bt)] = (Vt_b ,a_optimal)
    print(V_comp[t])
# print(V_table)
# print(Q_table)
pd.DataFrame.from_dict(V_comp, orient='index')





======Timestep : 5======
value solution = 79.99999890649667


value solution = 93.63636363636364
value solution = 79.96375351148667
value solution = 78.79674065019825
value solution = -1.463087248322148
value solution = -3.918614387215617
value solution = -3.997463118567846
{(0.9999999707097337, 2.9290266333289847e-08): (79.99999890649667, [0, 0, 0, 0, 0.9999999863312086, 0, 0, 1.3668791500152229e-08, 0]), (0.5, 0.5): (93.63636363636364, [0, 0, 0, 0.2727272727272727, 0, 0, 0.7272727272727273, 0, 0]), (0.9990311236573287, 0.0009688763426712272): (79.96375351148667, [0, 0, 0, 0, 0.9995473105008567, 0, 0, 0.0004526894991433942, 0]), (0.9697986577181208, 0.030201342281879186): (78.79674065019825, [0, 0, 0, 0, 0.9853541894988542, 0, 0, 0.014645810501145677, 0]), (0.030201342281879193, 0.9697986577181209): (-1.463087248322148, [0, 0, 0, 0, 1.0, 0, 0, 0, 0]), (0.0009688763426712277, 0.9990311236573288): (-3.918614387215617, [0, 0, 0, 0, 1.0, 0, 0, 0, 0]), (3.020096943040473e-05, 0.9999697990305696): (-3.997463118567846, [0, 0, 0, 0, 1.0, 0, 0, 0, 0])}
=

,0
6,0
5,"{(0.9999999707097337, 2.9290266333289847e-08):..."
4,"{(0.9999990594578604, 9.405421396307146e-07): ..."
3,"{(0.9999697990305696, 3.020096943040473e-05): ..."
2,"{(0.9990311236573288, 0.0009688763426712277): ..."
1,"{(0.9697986577181209, 0.030201342281879193): (..."


###


1. Cooperative game
2. zero sum
3. blind opponent -> Q(x,u) function only equal 
4. 


In [ ]:
V_table[1]

{(0.9697986577181209, 0.030201342281879193): (0.0,
  [0, 0, 0, 0, 0, 0.12904590380017084, 0, 0, 0.8709540961998292]),
 (0.5, 0.5): (-10.726677004999988, [1.0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (0.030201342281879193, 0.9697986577181209): (0.0,
  [0, 0, 0, 0, 0.8709540961998292, 0, 0, 0.12904590380017084, 0])}

## Check performance

### Deep Q Network implementation using LSTM recurrent Neural Networks

In [ ]:
import tensorflow

## SAMPLE BAG OF EXPERIENCES

In [ ]:
def closest_b(b,V_t):
    Bt = [np.array(i) for i in V_t.keys()]
    closest_bt = min(Bt, key=lambda x: np.linalg.norm(x-b))
    print(closest_bt)
    return tuple(closest_bt)

def sample_action(DR):
    action = np.random.choice(len(DR), 1, p=DR)[0]
    print(action)
    return prob.joint_actions[action]
        



In [ ]:
np.expand_dims(Q_table[1],axis=0)
np.matrix(Q_table[1])

matrix([[{(0.9697986577181209, 0.030201342281879193): [-8.466646064614089, -401.40426039597315, 12.018558395973173, -401.40426039597315, -202.23647516107388, -410.692851, 12.018558395973173, -410.692851, 60.850773161073846], (0.5, 0.5): [-10.726677004999988, -194.69285100000002, -194.692851, -194.69285100000002, -70.692851, -410.6928509999999, -194.692851, -410.6928509999999, -70.69285099999999], (0.030201342281879193, 0.9697986577181209): [-8.466646064614068, 12.018558395973173, -401.4042603959732, 12.018558395973173, 60.850773161073846, -410.692851, -401.4042603959732, -410.692851, -202.23647516107383]}]],
       dtype=object)

In [ ]:
prob.joint_actions

[('listen', 'listen'),
 ('listen', 'open-left'),
 ('listen', 'open-right'),
 ('open-left', 'listen'),
 ('open-left', 'open-left'),
 ('open-left', 'open-right'),
 ('open-right', 'listen'),
 ('open-right', 'open-left'),
 ('open-right', 'open-right')]

In [ ]:
np.tile((0.5,0.5), (9, 1))

array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]])

# DQN Design

LSTM --> varying input length 
RNN --> fixed input length

output heads -> should represent each action 

input of network should receive sequences of input observations sampled from the replay memory

hidden layers --> representative of belief state of the agent (may be wise to add more hidden layers )





In [ ]:
def convert(obs):
    indexes = []
    for z in obs:
        idx = prob.joint_observations.index(z)
        indexes.append(idx)
    return indexes


In [ ]:
# conda env config
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [ ]:
# import tensorflow as tf
# import numpy as np
# from collections import deque
# import random

# Environment parameters
num_states = 2
num_observations = 4
num_actions = 9

# DQN parameters
gamma = 0.9
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001
batch_size = 32
memory_size = 1000

In [ ]:
class Experience():
    def __init__(self,z,a,r,z_next): 
        self.observation = z 
        self.action = a
        self.reward = r
        self.next_observation = z_next
        self.weights_z = 0
        self.weights_z_next = 0
     
    

In [ ]:
import tensorflow as tf
import numpy as np
from collections import deque

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def add(self, belief, action,obs, reward, next_belief, done):
        self.buffer.append((belief, action,obs, reward, next_belief, done))
    def add_experience(self,experience):
        self.buffer.append(experience)

    def sample(self,batch_size):
        batch = np.random.choice(len(self.buffer), batch_size, replace=False)
        return batch
    
    # def sample(self, batch_size):
    #     batch = np.random.choice(len(self.buffer), batch_size, replace=False)
    #     b, a, r,obs, b_next, dones = zip(*[self.buffer[i] for i in batch])
    #     return np.array(b),np.array(obs), np.array(a), np.array(r, dtype=np.float32), np.array(b_next), np.array(dones, dtype=bool)
    
    def next(self,sample,t):
        b_next = sample[4]
        diff = np.inf
        next = None
        for i in self.buffer:
            if np.array_equal(i[0],sample[0]) and np.array_equal(i[4],b_next):
                pass
            val =  np.abs(np.linalg.norm(np.array(b_next)-np.array(i[0])))
            if  val<diff and val!=0:
                if i[5] == True and t<4:
                    pass
                diff=val
                next = i
        if next==None:
            print("experience not found")
        return next
        
    def initial_experience(self):
        for i in self.buffer:
            if np.array_equal(np.array(i[0]),np.array([0.5,0.5])) and i[5]!= True:
                return i
        return i
    def sample_sequences(self,batch_size,sequence_length):
        sequences = []
        for n in range(batch_size):
                batch = []
                e = self.initial_experience()
                batch.append(e)
                for t in range(sequence_length-1):
                    e = self.next(e,len(batch))
                    if type(e) == tuple:
                        batch.append(e)
                        if e[5]==True:
                            print(f"episode of length {t} found\n")
                            break
                
                # Use zip to transpose the input list of tuples
                transposed_tuples = zip(*batch)

                # Unpack the transposed result into individual lists
                b, a,obs, r, b_next, dones  = map(list, transposed_tuples)
             
                sequences.append([b,a,obs, r,b_next,dones])
        print(sequences)
        return sequences
    
    

In [ ]:
# new design implementations:

# each experience in the replay buffer should store e_t = <zt,at,rt,zt+1>
# feed neural network with single experiences from the replay memory
# NN weights at initial observation should always be the same, initialiw w_z0 
#dont forget to reset model to initial weigths when starting new episode


In [ ]:
class QNetwork(tf.keras.Sequential):
    def __init__(self, observation_size, action_size, hidden_size,sequence_length):
        super(QNetwork, self).__init__()
        self.lstm_layer = tf.keras.layers.LSTM(16, input_shape=(sequence_length,1), return_sequences=True)
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(36, activation='relu')
        self.dense2 = tf.keras.layers.Dense(action_size, activation='softmax')
        self.build(input_shape=(batch_size,sequence_length,1))



    def call(self, inputs):
        x = self.lstm_layer(inputs)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x
    
class DQNAgent:
    def __init__(self, observation_size, action_size, hidden_size, learning_rate=0.001, gamma=0.99, epsilon_decay=0.995, min_epsilon=0.01, replay_buffer_size=10000, batch_size=32):
        self.observation_size = 1
        self.action_size = action_size
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = 1.0
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.replay_buffer = ReplayBuffer(replay_buffer_size)
        self.batch_size = batch_size
        self.sequence_length = 5

        self.model = QNetwork(observation_size, action_size, hidden_size,self.sequence_length)
        self.target_model = QNetwork(observation_size, action_size, hidden_size,self.sequence_length)
        self.optimizer = tf.optimizers.Adam(learning_rate)
        
    def init_model(self):
        self.model.set_weights()

    def select_action(self, belief,t):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        belief = closest_b(belief,Q_table[t])
        q_values = Q_table[t][belief]
        return np.argmax(q_values)

    def update_epsilon(self):
        self.epsilon = max(self.epsilon * self.epsilon_decay, self.min_epsilon)

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def train(self):
        if len(self.replay_buffer.buffer) < self.sequence_length*self.batch_size/2:
            print("not enough experiences in REPLAY")
            return

        # Sample sequences from the replay buffer
        sequences =self.replay_buffer.sample_sequences(self.batch_size, self.sequence_length)

        # Initialize LSTM state
        lstm_state = None

        # print(sequences[:,2])
        
        with tf.GradientTape() as tape: 
            # Iterate over the sequence lengt
            obs = []
            next_obs = []
            for sequence in sequences:
                # Extract observations, actions, rewards, next_observations, and dones from the sequence
                beliefs, actions, observations, rewards, next_beliefs,done = map(list, sequence)

                
               

                observations = np.array(convert(observations),dtype=np.float32)
                print(f"observations :\n {observations}")
                print(f"rewards : \n{rewards}")
                obs.append(observations)
                next_obs.append(observations[1:])


           
            
            obs = np.array(obs).reshape(self.batch_size,self.sequence_length,1)
          
            next_obs = np.expand_dims(np.expand_dims(np.array(next_obs), axis=0), axis=0)
            # Predict Q-values for observations
            print(f"converted observations : \n{obs}")
            q_values = self.model(obs)
            print(f"model output\n: {q_values}")
            # Reshape to (batch_size, action_size) for indexing
            q_val = q_values.reshape((self.batch_size, self.action_size, -1))
            print(f"model output reshape\n: {q_values}")
                # # Find the maximum Q-value for each next belief
                # max_next_q_values = np.max(next_q_values, axis=1)

                # # Calculate target Q-values using the target network
                # target_q_values = self.target_model.predict(next_obs)

                # # Update target Q-values based on the SARSA algorithm
                # target_q_values[range(self.batch_size), actions] = rewards + (1 - dones) * self.gamma * max_next_q_values

                # # Use the current Q-network to calculate Q-values for the batch of current observations
                # q_values = self.model.predict(observations, initial_state=lstm_state)

                # # Calculate the loss using mean squared error
                # action_masks = tf.one_hot(actions, self.action_size)
                # selected_q_values = tf.reduce_sum(q_values * action_masks, axis=1)
                # loss = tf.losses.mean_squared_error(target_q_values, selected_q_values)

                # # Compute gradients for this timestep
                # gradients = tape.gradient(loss, self.model.trainable_variables)

                # # Apply gradients for this timestep
                # self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

                # # Update LSTM state for the next timestep
                # lstm_state = self.model.lstm_layer(observations, initial_state=lstm_state, training=True)


def main():
    # Environment parameters
    observation_size = 1
    action_size = 9

    # Agent parameters
    hidden_size = 2
    learning_rate = 0.001
    gamma = 0.99
    epsilon_decay = 0.995
    min_epsilon = 0.01
    replay_buffer_size = 10000
    batch_size = 5
    sequence_length = 6

    # Create DQNAgent
    agent = DQNAgent(num_observations, num_actions, hidden_size, learning_rate, gamma, epsilon_decay, min_epsilon, replay_buffer_size, batch_size)

    # Training loop
    episodes = 1000

    for episode in range(episodes):
        belief = [0.5,0.5]  # Initial belief state
        last_obs = False
        prob.reset()
        hidden_state = 0
        init_weights = agent.model.get_weights()
        for step in range(sequence_length):  # Assume a maximum of 6 steps per episode
            action = agent.select_action(belief,step)
            
            # Simulate environment and get next observation, reward, done
            next_obs, reward , done , _ = prob.step(prob.joint_actions[action])

            # get hidden layer activation
            weights = agent.model.get_weights()
            # record experience
            experience = Experience(last_obs, weights, action,reward,next_obs)

            # add experience to replay memory
            agent.replay_buffer.add_experience(experience)
            #agent.replay_buffer.add(belief, action, obs, reward, b_next, done)

            #calculate next belief
            b_next = next_belief(belief,def_actions[action],prob.joint_observations.index(obs))

        
            belief =b_next
            last_obs = next_obs
            if done:
                break
        
        # Update target network periodically
        if episode % 10 == 0:
            agent.update_target_model()
            agent.train()

        # Update epsilon
        agent.update_epsilon()

main()


TypeError: Experience.__init__() takes 5 positional arguments but 6 were given

In [ ]:
prob.transition_fn

array([[[1. , 0. ],
        [0. , 1. ]],

       [[0.5, 0.5],
        [0.5, 0.5]],

       [[0.5, 0.5],
        [0.5, 0.5]],

       [[0.5, 0.5],
        [0.5, 0.5]],

       [[0.5, 0.5],
        [0.5, 0.5]],

       [[0.5, 0.5],
        [0.5, 0.5]],

       [[0.5, 0.5],
        [0.5, 0.5]],

       [[0.5, 0.5],
        [0.5, 0.5]],

       [[0.5, 0.5],
        [0.5, 0.5]]])

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()